In [ ]:
import threading
import openai
import requests
import json
import os
import time
import pandas as pd
import openpyxl
from datetime import datetime


openai.api_key = ""
openrouter_key = ""

In [ ]:
ai_models = [
    "gpt-4",
    "gpt-3.5-turbo",
    "anthropic/claude-2",
    "meta-llama/llama-2-13b-chat",
    "huggingfaceh4/zephyr-7b-beta",
]

temperature = 0.00

In [ ]:
instructions = [
    "Build a 550 resistor using other resistors.",
    "Build a voltage divider where Vin is 10V and Vout to be 4V.",
    "Build a RC high-pass filter with a cutoff of 10 kHz.",
    "Build a circuit for a three input NAND gate using CMOS logic.",
    "Build a 1 of 4 decoder only using logic gates.",
    "Build me a 3 bit ripple counter circuit.",
    "Build a circuit that can convert 15V to 5V for 1A using a IC voltage regulator.",
    "Build a square wave frequency generator at 10kHz using a NE555.", 
    "Build a circuit that achieves a non-inverting voltage gain of 5 using an op-amp LM741.",
    "Build me a circuit with an Arduino Uno R3 (ATMEGA328P) using the MCP4921 DAC and BME280 sensor."
]

In [ ]:
def get_api_output(model, temperature, system_prompt, user_prompt):
    if model == 'gpt-4' or model == 'gpt-3.5-turbo' or model == 'gpt-4-1106-preview':
        
        completion = openai.ChatCompletion.create(
            model=model,
            temperature=temperature,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )
        api_output = completion.choices[0].message.content
        return api_output
    
    else:
        response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",
            headers={
                "Authorization": 'Bearer ' + openrouter_key,  
                "Content-Type": "application/json"
            },
            data=json.dumps({
                "model": model,
                'temperature': temperature,
                'max_tokens': 1024,
                "messages": [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ]
            })
        )
        #print(response.text)
        data = json.loads(response.text)
        output_string = data['choices'][0]['message']['content']
        return output_string

In [ ]:
def extract_and_execute_code(result):
    # Markers for the Python code block
    start_marker = "```python"
    end_marker = "```"

    # Check start and end markers
    start_index = result.find(start_marker)
    end_index = result.find(end_marker, start_index + len(start_marker))

    local_namespace = {}

    # Executing the text only if both markers are found
    if start_index != -1 and end_index != -1:
        # Adjusting start index to get the actual start of the code
        start = start_index + len(start_marker)
        
        # Extract the Python code
        python_code = result[start:end_index].strip()

        try:
            # Execute the script
            exec(python_code, {}, local_namespace)
        except Exception as e:
            return "Script Error"

    else:
        try:
            # Execute the script
            exec(result, {}, local_namespace)
        except Exception as e:
            return "Script Error"

    # Retrieve the variable
    partlist = local_namespace.get('partlist', 'Variable not found')

    return partlist

In [ ]:
def get_string_after_slash(input_string):
    # Split the string by the slash character
    parts = input_string.split('/')

    # Return the part after the last slash
    # If there is no slash, return the entire string or an empty string
    return parts[-1] if parts else input_string

In [ ]:
class NetlistCreator:

    def __init__(self, input_string):
        self.input_string = input_string
        self.node_list = []
        self.node_list.append(['GND'])
        self.netlist = ""

    
    def sorting_key(string):
        if string.startswith("INPUT"):
            return (0, string)  
        elif string.startswith("OUTPUT"):
            return (1, string)  
        elif string.startswith("POWER"):
            return (2, string)  
        else:
            return (3, string)  
    
    def similar_then_combine(self,list1, list2):
        set1 = set(list1)
        set2 = set(list2)
        if set1.intersection(set2):
            combined_list = [item for item in set1.union(set2)]
            combined_list = sorted(combined_list, key=NetlistCreator.sorting_key)
            return True, combined_list
        return False, list2
    
    def check_prefix_and_return(self, input_list, item_to_return):
        if input_list[0].startswith("INPUT"):
            parts = input_list[0].split('_')
            output = 'IN' + parts[1]
            return output
        elif input_list[0].startswith("OUTPUT"):
            parts = input_list[0].split('_')
            output = 'OUT' + parts[1]
            return output
        elif input_list[0].startswith("POWER"):
            parts = input_list[0].split('_')
            output = 'PWR' + parts[1]
            return output
        else:
            return item_to_return
    
    def append_list(self, list_to_append):
        for index, inner_list in enumerate(self.node_list):
            if inner_list == list_to_append:
                return self.check_prefix_and_return(inner_list, index)
            similar, list_to_append = self.similar_then_combine(inner_list, list_to_append)
            if similar:
                self.node_list[index] = list_to_append
                return self.check_prefix_and_return(list_to_append, index)
        self.node_list.append(list_to_append)
        return self.check_prefix_and_return(list_to_append, len(self.node_list)-1)

    def create_netlist(self):
        index = 0
        check = True
        device_name = ''
        lines = self.input_string.strip().split('\n')

        # Function to find the index of the line containing "Connections:"
        def find_connections_index(string_list):
            for index, line in enumerate(string_list):
                if "Connections:" in line:
                    return index+1
            return -1  # Return -1 if not found
        
        index = find_connections_index(lines)

        if index == -1:
            return ''

        while index < len(lines):
            line = lines[index]
            if line.strip() == '':
                self.netlist += '\n'
                check = True
            else:
                if check:
                    device_name = line.split()
                    split_strings = line.split(" ", 1)
                    device_name = split_strings[1] + ' '
                    self.netlist += device_name 
                    device_name = device_name[:-2]
                    check = False
                else:
                    split_line = line.strip().split(':')
                    pin_info = line.strip().split(':')[0]
                    pin_key = device_name + '_' + pin_info
                    if len(split_line) > 1:
                        pin_connections = [item.strip() for item in split_line[1].split(',')]
                        if 'GND' in pin_connections[0]:
                            self.netlist += 'GND'
                            self.netlist += ' '
                        elif 'NC' in pin_connections[0]:
                            self.netlist += '~' 
                            self.netlist += ' '
                        else:
                            pin_connections.append(pin_key)
                            pin_connections = sorted(pin_connections, key=NetlistCreator.sorting_key)
                            pin_connections = [s.replace(" ", "").replace("\t", "") for s in pin_connections]
                            node_num = self.append_list(pin_connections)
                            self.netlist += str(node_num) 
                            self.netlist += ' '
            index += 1


        lines = self.netlist.splitlines()
        non_empty_lines = [line for line in lines if line.strip()]
        self.netlist = '\n'.join(non_empty_lines)
        
        # Apply modifications so that the parts will be sort by highest pin order to lowest
        # and also have the pin number at the end.

        # Make into a list again
        new_lines = self.netlist.split('\n')

        # Reusing the function to count items between spaces
        def count_items_in_line(string):
            items = [item for item in string.split(' ') if item]
            return len(items)

        # Modify each line by adding count and underscore
        def modify_new_line(line):
            parts = line.split(':')
            count = count_items_in_line(parts[1])
            new_line = f"{parts[0]}: {parts[1]}"
            return new_line

        # Apply modifications to each line
        modified_new_lines = [modify_new_line(line) for line in new_lines]

        # Sorting the modified lines by item count in descending order
        lines_with_counts_new = [(line, count_items_in_line(line.split(':')[1])) for line in modified_new_lines]
        sorted_new_lines = sorted(lines_with_counts_new, key=lambda x: x[1], reverse=True)

        # Extracting the sorted lines
        final_sorted_lines = [line[0] for line in sorted_new_lines]

        # Convert to 
        self.netlist = "\n".join(final_sorted_lines)

        return self.netlist
    

    def print_netlist_details(self):
        print("\n===Input Connections===")
        print(self.input_string)

        print("\n===Netlist===")
        print(self.netlist)

        print("\n===Node lists===")
        for index, list in enumerate(self.node_list):
            print(f"Node Number: {index} - {list}")

In [ ]:
Component_Pinouts = {
"resistor": 
"""
R#_#
    1: 
    2: 
""",
"inductor": 
"""
L#_#
    1: 
    2: 
""",
"capacitor": 
"""
C#_#
    1: 
    2: 
""",
"AND": 
"""
U#_ANDGate:
    1_A:  
    2_B: 
    3_OUT: 
""",
"NOT": 
"""
U#_NOTGate:
    1_IN:   
    2_OUT: 
""",
"PMOS": 
"""
U#_PMOS:
    1_Gate:
    2_Drain:
    3_Source: 
""",
"NMOS":  
"""
U#_NMOS:
    1_Gate:
    2_Drain:
    3_Source:
""",
"74LS08":  # AND Gate: 1 of 4 decoder
"""
U#_74LS08:
    1_A1:
    2_B1:
    3_Y1: 
    4_A2:
    5_B2:
    6_Y2:
    7_GND: 
    8_Y3:
    9_A3:
    10_B3: 
    11_Y4:
    12_A4:
    13_B4: 
    14_VCC: 
""",
"74LS00":  # NAND Gate: 1 of 4 decoder
"""
U#_74LS00:
    1_A1:
    2_B1:
    3_Y1: 
    4_A2:
    5_B2:
    6_Y2:
    7_GND: 
    8_Y3:
    9_A3:
    10_B3: 
    11_Y4:
    12_A4:
    13_B4: 
    14_VCC: 
""",
"74LS04":  # NAND Gate: 1 of 4 decoder
"""
U#_74LS00:
    1_A1:
    2_Y1:
    3_2A: 
    4_2Y:
    5_3A:
    6_3Y:
    7_GND: 
    8_4Y:
    9_4A:
    10_5Y: 
    11_5A:
    12_6Y:
    13_6A: 
    14_VCC: 
""",
"JK": 
"""
U#_JKFlipFlop:
    1_J: 
    2_K: 
    3_Q: 
    4_Qbar: 
    5_CLK: 
""",
"74LS74": # JK FF: Ripple Counter
"""
U#_74LS74:
    1_CLR: 
    2_1D: 
    3_1CLK:
    4_1PK:
    5_1Q:
    6_1QBAR:
    7_GND:
    8_2QBAR:
    9_2Q:
    10_2PR:
    11_2CK:
    12_2D:
    13_2CLR:
    14_VCC:
""",
"LM7805": 
"""
U#_LM7805:
    1_INPUT: 
    2_GND: 
    3_OUTPUT:
""",
"NE555": 
"""
U#_NE555:
    1_GND: 
    2_TRIG:
    3_OUT: 
    4_RESET: 
    5_CTRL: 
    6_THR: 
    7_DIS: 
    8_VCC: 
""",
"LM741":
"""
U#_LM741:
    1_OFFSET: 
    2_INVERTING:
    3_NON-INVERTING: 
    4_V-: 
    5_OFFSET: 
    6_OUTPUT: 
    7_V+: 
    8_NC: 
""",
"MCP4921":
"""
U#_MCP4921:
    1_VDD: 
    2_CS:
    3_SCK: 
    4_SDI: 
    5_LDAC: 
    6_VREF: 
    7_VSS: 
    8_VOUT: 
""",
"BME280":
"""
U#_MCP4921:
    1_GND: 
    2_CSB:
    3_SDI: 
    4_SCK: 
    5_SDO: 
    6_VDDIO: 
    7_GND: 
    8_VDD: 
""",
"R3":
"""
U#_ARDUINOUNO:
    1_NC: 
    2_IOREF: 
    3_RESET: 
    4_3V3: 
    5_5V: 
    6_GND: 
    7_GND: 
    8_VIN: 
    9_A0: 
    10_A1: 
    11_A2: 
    12_A3: 
    13_A4: 
    14_A5: 
    15_D0:
    16_D1
    17_D2:
    18_D3:
    19_D4:
    20_D5:
    21_D6:
    22_D7:
    23_D8:
    24_D9:
    25_SS:
    26_MOSI:
    27_MISO:
    28_SCK:
    29_GND:
    30_AREF:
    31_SDA:
    32_SCL:
""",      
}

In [ ]:
system_role_partlist2 = '''
You are a helpful assistant for creating a python script that will calculate component or component values and print a Part List for an electronic design.

The part list should detail each component's value and name, quantity, and purpose in the circuit. 

"Part List" section:
- Part Name can be resistor capacitor, inductors, or any other parts such as ICs.
- Only include the minimal parts needed to build the basic circuit and DO not include any other parts that don't have a purpose.
- The Purpose section of the part list should explain the purpose of that part.
- If a IC is requested try to pick a well known IC to put in the Part List.
- Do NOT include any Power Supply or Ground or wires or a bread board in the Part List. 
- Do NOT include additional explanations, methods, diagrams, or theoretical background.

Here is an example format of your output that should be formatted as provided below:
Part List: 
1uF Capacitor | Quantity: 1 | Purpose: ...
ABC123 Imaginary IC | Quantity: 1 | Purpose: ...
'''

In [ ]:
system_role_partlist3 = '''
You are a helpful assistant for creating a python script that will calculate component or component values and print a Part List for an electronic design.

Script Structure: The script should have clear sections for defining component values, performing the calculation, and outputting the part list.
Part List Format: The part list should detail each component's value and name, quantity, and purpose in the circuit. 
For example: 1uF Capacitor | Quantity: 1 | Purpose: ...
For example: ABC123 Imaginary IC | Quantity: 1 | Purpose: ...

ONLY give 1 complete and valid syntax Python Script with the printed Part List in the script.
Do NOT include additional explanations, how the script works, output of the script, methods, diagrams, or theoretical background.

"Define Values" section:
- Should only define values needed for the calculations.
- All variables should be predefined or initialized to zero or 0 to prevent syntax errors.
- In this section Do NOT include SI units for the values such as K, M or other letters or SI symbols in the "Define Values" of the code.
For example: resistor value in the python script should say 1000 instead of 1k. 

"Calculations" section:
- Find a appropiate equation from the user's prompt 
- Derive the equation such that values can be plugged into the variables to calculate the component value/s.
- Do NOT include any calculations if based on the user prompt it will not require any calculations. 
- Do NOT multiply or divide with any negiative numbers.

"Part List Output" section:
- Part Name can be resistor capacitor, inductors, or any other parts such as ICs.
- Only include the minimal parts needed to build the basic circuit and DO not include any other parts that don't have a purpose.
- The Purpose section of the part list should explain the purpose of that part.
- If a IC is requested try to pick a well known IC to put in the Part List.
- Do NOT give me the values of the components without calculating them first.
- Do NOT include any Power Supply or Ground or wires or a bread board in the Part List. 

Here is an example format of your output that should be formatted as provided below:
```python 
# Define Values:
requirement_value = # add number
other_part_value = # add number
# .. More if needed

# Calculations: 
# This sections should start with an equation and derive it. Then plug in variables to calculate the value/s.

# Part List Output:
partlist = f"""
Part List:
{value} Part Name |  Quantity: 1 | Purpose: ...  
Another Part Name |  Quantity: 1 | Purpose: ...   
# .. More if needed
"""

print(partlist)
```
'''

In [ ]:
system_prompt_netlist = """
You are a helpful assistant for providing and making connections from the User's prompt from the components in the "Part List."

Tell me what node connections there would be with the parts from the Part List and include pins names with the part.
Keep in mind GND/Ground should never be connected directly to Power Connection such as VDD or VCC and that there should also be a part connected between both.
If every part's pins are unconnected, this is wrong you should fix this before giving me the list.
Pin names almost should never be NC (not connected), so do not name pins with NC.
For the resistors, capacitors, or inductors, they will all have 2 pins, pin 1: cathode and pin2: anode. 
List the resistors, capacitors, and inductors in seqeuence C1 C2 .. R1, R2, L1, L2 and so on.

Any inputs should be labeled INPUT_# where # is the number of that input out of all inputs.
Any outputs should be labeled OUTPUT_# where # is the number of that output out of all outputs.
Any power rails or connections should be denoted as POWER_# where # is the voltage of that rail.

Make sure to list ALL connections for the pin if they are connected to it except for NC and GND.
Make sure that the values from the user given Part List section are the same values with SI prefixes that you give in the "Connections" section for components.
You must list every part that is in the Part List and ONLY list that amount OR quantity given.

Any ICs, should be given a U# where # is the number in sequence of that IC. There should be no U# that have the same number.

Do NOT include additional explanations, how the script works, output of the script, methods, diagrams, or theoretical background.

Below after the ':' is the output example.:
Connections:
    U1_ABC123:
        1_VCC: POWER_5V
        2_IN: INPUT_1, R1_10k_2, R2_10k_1
        3_CONTROL: NC

    R1_10k:
        1: POWER_10V
        2: INPUT_1, U1_ABC123_2_IN1, R2_10k_1

    R2_10k:
        1: INPUT_1, U1-ABC123_2_IN1, R1-10k_2
        2: GND

    C1_1uH:
        1: POWER_5V
        2: GND

    L1_100uH:
        1: U1_ABC123_2_IN
        2: OUTPUT_1
"""

In [ ]:
def getPartNames(input_string):

    # Splitting the string into lines
    lines = input_string.split('\n')
    
    # Initializing the output list
    parts = []

    for line in lines:
        # Skip the 'Partlist:' line
        if 'Part List:' in line:
            continue
        
        # Splitting each line at '|', and taking the first part
        
        part = line.split('|')[0].strip()
        
        # Adding to the list if not empty
        if part:
            parts.append(part)
    
    # Joining the parts into a single string
    return parts

In [ ]:
def searchComponentPinouts(strings, dictionary):
    # Initialize a set to store unique matches
    matches = set()

    lowercase_keys_dict = {key.lower(): key for key in dictionary}

    for string in strings:
        words = string.split()

        for word in words:
            min_length = 2 if len(word) < 3 else 3

            for i in range(len(word)):
                for j in range(i + min_length, len(word) + 1):
                    substring = word[i:j].lower()

                    if substring in lowercase_keys_dict:
                        matches.add(lowercase_keys_dict[substring])

    return matches

In [ ]:
# Proposed Pipeline System
model_name = ai_models[3]
name = get_string_after_slash(model_name)

current_time_numerical = datetime.now().strftime("%Y%m%d%H%M%S")

instruction_index = 4
instruction = instructions[instruction_index]

filename = f"Tests/{name}_{current_time_numerical}.xlsx"

# Create a new workbook
with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        
    for instruction_index, instruction in enumerate(instructions):

        results = []
        for i in range(10):

            print(f"=== {name} - instruction: {instruction} - step: {i} ===")
            
            time.sleep(3) 

            try:
                description_result = get_api_output(model_name, temperature, system_role_partlist2, instruction)
            except Exception as e:
                description_result = "error"

            time.sleep(3) 

            try:
                connections_result = get_api_output(model_name, temperature, system_prompt_netlist, instruction + '\n' + description_result)
            except Exception as e:
                connections_result = "error"
                
            # Create Netlist
            netlist_creator = NetlistCreator(connections_result)
            print("====connections_result====")
            print(connections_result)
            
            try:
                netlist = netlist_creator.create_netlist()
            except Exception as e:
                netlist = "error"

            print("====netlist====")
            print(netlist)

            results.append(["0", description_result, connections_result, netlist, 'U', 'U'])

        sheet_num = instruction_index+1
        df = pd.DataFrame(results, columns=['Temperature', 'Description Output', 'Connections Output', 'Connections Netlist Output', 'Correct Parts', 'Correct Connections'])
        df.to_excel(writer, sheet_name=f'Sheet{sheet_num}', index=False)


In [ ]:
# Proposed Pipeline System with Additional Features

# uncommment for looping
#for model_name in ai_models: 
model_name = ai_models[2]
name = get_string_after_slash(model_name)

current_time_numerical = datetime.now().strftime("%Y%m%d%H%M%S")

instruction_index = 4
instruction = instructions[instruction_index]

filename = f"Tests7/{name}_{instruction_index}_{current_time_numerical}.xlsx"

# Create a new workbook
with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        
    for instruction_index, instruction in enumerate(instructions):

        results = []
        for i in range(10):

            print(f"=== {name} - instruction: {instruction} - step: {i} ===")
            
            #time.sleep(3) 

            # Get Part List 
            try:
                partlist_result = get_api_output(model_name, temperature, system_role_partlist3, instruction)
                partlist_output = extract_and_execute_code(partlist_result)
            except Exception as e:
                partlist_result = 'error'
                partlist_output = "error"
                
            #time.sleep(3) 
            
            if str(partlist_output).startswith("Script Error") or str(partlist_output).startswith("error"):
                results.append(["0", partlist_result, partlist_output, '0', '0', 'U', 'U'])

            # Get Connections    
            else:       
                # Getting all unique matches
                partnames = getPartNames(partlist_output)
                matches = searchComponentPinouts(partnames, Component_Pinouts)

                # Append the pinout of component for next stage
                out = "Device Pinouts:\n"
                for word in matches:
                    out += Component_Pinouts[word]

                user_prompt = instruction + "\n" + partlist_output + "\n" + out
                print(user_prompt)

                try:
                    connections_result = get_api_output(model_name, temperature, system_prompt_netlist, user_prompt)
                except Exception as e:
                    netlist = "error"
                

                print("====")
                # Create Netlist
                netlist_creator = NetlistCreator(connections_result)
                print(connections_result)
                print("====")
                
                try:
                    netlist = netlist_creator.create_netlist()
                except Exception as e:
                    netlist = "error"
                    
                print(netlist)

                results.append(["0", partlist_result, partlist_output, connections_result, netlist, 'U', 'U'])

        sheet_num = instruction_index+1
        df = pd.DataFrame(results, columns=['Temperature', 'Part List Code', 'Part List Output', 'Connections Output', 'Connections Netlist Output', 'Correct Parts', 'Correct Connections'])
        df.to_excel(writer, sheet_name=f'Sheet{sheet_num}', index=False)
